In [1]:
import pandas as pd
import dash
from dash import dcc
from dash import html
import plotly.graph_objs as go

In [2]:
# Data Preprocessing

df = pd.read_csv('dataset_space_API_past_launches_raw.csv')

df = df.drop(['FlightNumber', 'Date', 'Longitude', 'Latitude'], axis=1)

# Encode Outcome
df = df.replace({'Outcome': {'True Ocean': 1, 
                             'True RTLS': 1,
                             'True ASDS': 1,
                             'False Ocean':0,
                             'False RTLS':0,
                             'False ASDS':0,
                             'None ASDS':0,
                             'None None':0,},
                 'GridFins': {True: 1, False: 0},
                 'Reused': {True: 1, False: 0},
                 'Legs': {True: 1, False: 0}
                })

# Missing Values
PayloadMass_mean = df['PayloadMass'].mean()
PayloadMass_mean = round(PayloadMass_mean, 1)
df['PayloadMass'] = df['PayloadMass'].fillna(PayloadMass_mean)
df['LandingPad'] = df['LandingPad'].fillna('missing')
df['Block'] = df['Block'].fillna('0')

In [ ]:
data = df.groupby('LaunchSite')['Outcome']
data = data.agg([('success', 'sum'), ('failure', lambda x: len(x) - sum(x))])
data = data.to_dict(orient='index')

app = dash.Dash(__name__)
options = [{'label': key.upper(), 'value': key} for key in data.keys()]
app.layout = html.Div(children=[
    html.H1(children='Success Rate of LaunchSite'),
    dcc.Dropdown(
        id = 'dropdown',
        options = options,
        value = options[0]['value']
    ),
    dcc.Graph(id = 'graph')
])

@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('dropdown', 'value')]
)

def update_graph(selected_value):
    values = data[selected_value]
    chart = go.Figure(
        data = [go.Pie(labels = list(values.keys()), values = list(values.values())
        )]
    )
    return chart

if __name__ == '__main__':
    app.run_server(debug = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Jun/2023 19:43:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 19:43:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 19:43:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 19:43:17] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 19:43:17] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 19:43:17] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 19:43:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2023 19:43:19] "POST /_dash-update-component HTTP/1.1" 200 -
